### Challenge 14:  Byte-at-a-time ECB decryption (Harder)

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

Take your oracle function from [Challenge 12](12.ipynb). Now generate a random count of random bytes and prepend this string to every plaintext. You are now doing:

```AES-128-ECB(random-prefix || attacker-controlled || target-bytes, random-key)```
    
Same goal: decrypt the target-bytes.    
    
</div>

In [1]:
import base64
import cryptopals as cp
from numpy.random import randint

# ---------UNKNOWN PARAMETERS ------------
unknown_key = bytes(list(randint(0, 17, 16)))
random_prefix = bytes(list(randint(0, 256, randint(0, 16))))
# ---------END UNKNOWN PARAMETERS---------

In [2]:
def encryption_oracle_3(data, key, random_prefix):

    unknown_string = "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc\
                      28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZG\
                      J5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5\
                      vLCBJIGp1c3QgZHJvdmUgYnkK"

    AES_input = random_prefix + data + base64.b64decode(unknown_string)

    return(cp.AESEncrypt(AES_input, key))

<div class="alert alert-block alert-info">
  
### Stop and think for a second.

What's harder than challenge #12 about doing this? How would you overcome that obstacle? The hint is: you're using all the tools you already have; no crazy math is required.

Think "STIMULUS" and "RESPONSE".
    
</div>

In [3]:
plaintext = b''

# First step... find length of the random prefix.
# Assume we already know the oracle is using ECB), and block size is 16
# Feed it 2 identical blocks plus incrementally add one more byte that's
# different from the byte used for those until we detect two identical 
# (adjacent) ciphertext blocks 

probe_data = (b'\xff' * 32)
block_size_found = False

byte_ctr = 0
while not(block_size_found):
    
    ciphertext = encryption_oracle_3(probe_data, unknown_key, random_prefix)
    
    # We'll use a slightly different ECB detector -- I want to identify any 
    # adjacent identical blocks of ciphertext...
    ECB_block_idx = cp.detect_AES_ECB_adjacent(ciphertext)
    
    if ECB_block_idx >= 0:
        block_size_found = True
    elif byte_ctr > 16:
        print('Not able to find padding that led to ECB detect!')
        break
    else:
        byte_ctr += 1
        probe_data = b'\x00' + probe_data
    
print(f"Random prefix length is: {len(random_prefix)}")
print(f"Probe data length is: {len(probe_data)}")

Random prefix length is: 3
Probe data length is: 45


In [10]:
block_idx_offset = ECB_block_idx + 2
for block_idx in range(1,10):
    
    for ii in range(15, -1, -1):

        malicious_data = probe_data + (b'\x00' * ii)
        output_to_match = cp.encryption_oracle_3(malicious_data, unknown_key, random_prefix)
        malicious_data += plaintext
        #pdb.set_trace()
        Found = False
        
        start_idx = block_idx + block_idx_offset
        
        for chr_idx in range(256):
            # Try a character
            try_data = malicious_data + bytes([chr_idx])
            # print(try_data)
            if  cp.encryption_oracle_3(try_data, unknown_key, random_prefix)[0:(start_idx*16)-1] == output_to_match[0:(start_idx*16)-1]:
                Found = True
                #print(f'Found match! Byte value is:  {chr_idx}')
                #print(try_data)
                plaintext += bytes([chr_idx])
                break
                
        if Found == False:
            print(f"No match found at block_idx: {block_idx}, chr_idx: {ii}")
            break
            
print()            
print('*******************')
print('Decoded the secret message!  Plaintext is:\n')
print(plaintext.decode())

No match found at block_idx: 9, chr_idx: 4

*******************
Decoded the secret message!  Plaintext is:

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)